In [1]:
!pip install -U xgboost -f /kaggle/input/xgboost-python-package/ --no-index

Looking in links: /kaggle/input/xgboost-python-package/


In [2]:
#General
import pandas as pd
import numpy as np
import json

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore, Style, init;

# Modeling
import xgboost as xgb
import lightgbm as lgb
import torch
import sklearn 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

# Geolocation
from geopy.geocoders import Nominatim

# Options
pd.set_option('display.max_columns', 100)

In [3]:
DEBUG = True 

In [4]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [5]:
# Helper functions
def display_df(df, name):
    '''Display df shape and first row '''
    PrintColor(text = f'{name} data has {df.shape[0]} rows and {df.shape[1]} columns. \n ===> First row:')
    display(df.head(1))

# Color printing    
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    '''Prints color outputs using colorama of a text string'''
    print(style + color + text + Style.RESET_ALL); 

In [ ]:
DATA_DIR = "/kaggle/input/predict-energy-behavior-of-prosumers/"

# Read CSVs and parse relevant date columns
train = pd.read_csv(DATA_DIR + "train.csv")
client = pd.read_csv(DATA_DIR + "client.csv")
historical_weather = pd.read_csv(DATA_DIR + "historical_weather.csv")
forecast_weather = pd.read_csv(DATA_DIR + "forecast_weather.csv")
electricity = pd.read_csv(DATA_DIR + "electricity_prices.csv")
gas = pd.read_csv(DATA_DIR + "gas_prices.csv")

In [ ]:
location = (pd.read_csv("/kaggle/input/fabiendaniels-mapping-locations-and-county-codes/county_lon_lats.csv")
            .drop(columns = ["Unnamed: 0"])
           )

In [8]:
display_df(train, 'train')
display_df(client, 'client')
display_df(historical_weather, 'historical weather')
display_df(forecast_weather, 'forecast weather')
display_df(electricity, 'electricity prices')
display_df(gas, 'gas prices')
display_df(location, 'location data')

train data has 2018352 rows and 9 columns. 
 ===> First row:


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0


client data has 41919 rows and 7 columns. 
 ===> First row:


,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2


historical weather data has 1710802 rows and 18 columns. 
 ===> First row:


...

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1


gas prices data has 637 rows and 5 columns. 
 ===> First row:


,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1


location data data has 75 rows and 3 columns. 
 ===> First row:


,county,longitude,latitude
0,0,24.2,59.1


In [9]:
# See county codes
with open(DATA_DIR + 'county_id_to_name_map.json') as f:
    county_codes = json.load(f)
pd.DataFrame(county_codes, index=[0])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,HARJUMAA,HIIUMAA,IDA-VIRUMAA,JÄRVAMAA,JÕGEVAMAA,LÄÄNE-VIRUMAA,LÄÄNEMAA,PÄRNUMAA,PÕLVAMAA,RAPLAMAA,SAAREMAA,TARTUMAA,UNKNOWN,VALGAMAA,VILJANDIMAA,VÕRUMAA


In [ ]:
class FeatureProcessorClass():
    def __init__(self):         
        # Columns to join on for the different datasets
        self.weather_join = ['datetime', 'county', 'data_block_id']
        self.gas_join = ['data_block_id']
        self.electricity_join = ['datetime', 'data_block_id']
        self.client_join = ['county', 'is_business', 'product_type', 'data_block_id']
        
        # Columns of latitude & longitude
        self.lat_lon_columns = ['latitude', 'longitude']
        
        # Aggregate stats 
        self.agg_stats = ['mean'] #, 'min', 'max', 'std', 'median']
        
        # Categorical columns (specify for XGBoost)
        self.category_columns = ['county', 'is_business', 'product_type', 'is_consumption', 'data_block_id']

    def create_new_column_names(self, df, suffix, columns_no_change):
        '''Change column names by given suffix, keep columns_no_change, and return back the data'''
        df.columns = [col + suffix 
                      if col not in columns_no_change
                      else col
                      for col in df.columns
                      ]
        return df 

    def flatten_multi_index_columns(self, df):
        df.columns = ['_'.join([col for col in multi_col if len(col)>0]) 
                      for multi_col in df.columns]
        return df
    
    def create_data_features(self, data):
        '''📊Create features for main data (test or train) set📊'''
        # To datetime
        data['datetime'] = pd.to_datetime(data['datetime'])
        
        # Time period features
        data['date'] = data['datetime'].dt.normalize()
        data['year'] = data['datetime'].dt.year
        data['quarter'] = data['datetime'].dt.quarter
        data['month'] = data['datetime'].dt.month
        data['week'] = data['datetime'].dt.isocalendar().week
        data['hour'] = data['datetime'].dt.hour
        
        # Day features
        data['day_of_year'] = data['datetime'].dt.day_of_year
        data['day_of_month']  = data['datetime'].dt.day
        data['day_of_week'] = data['datetime'].dt.day_of_week
        
        return data

    def create_client_features(self, client):
        '''💼 Create client features 💼'''
        # Modify column names - specify suffix
        client = self.create_new_column_names(client, 
                                           suffix='_client',
                                           columns_no_change = self.client_join
                                          )       
        return client
    
    def create_historical_weather_features(self, historical_weather):
        '''⌛🌤️ Create historical weather features 🌤️⌛'''
        
        # To datetime
        historical_weather['datetime'] = pd.to_datetime(historical_weather['datetime'])
        
        # Add county
        historical_weather[self.lat_lon_columns] = historical_weather[self.lat_lon_columns].astype(float).round(1)
        historical_weather = historical_weather.merge(location, how = 'left', on = self.lat_lon_columns)

        # Modify column names - specify suffix
        historical_weather = self.create_new_column_names(historical_weather,
                                                          suffix='_h',
                                                          columns_no_change = self.lat_lon_columns + self.weather_join
                                                          ) 
        
        # Group by & calculate aggregate stats 
        agg_columns = [col for col in historical_weather.columns if col not in self.lat_lon_columns + self.weather_join]
        agg_dict = {agg_col: self.agg_stats for agg_col in agg_columns}
        historical_weather = historical_weather.groupby(self.weather_join).agg(agg_dict).reset_index() 
        
        # Flatten the multi column aggregates
        historical_weather = self.flatten_multi_index_columns(historical_weather) 
        
        # Test set has 1 day offset for hour<11 and 2 day offset for hour>11
        historical_weather['hour_h'] = historical_weather['datetime'].dt.hour
        historical_weather['datetime'] = (historical_weather
                                               .apply(lambda x: 
                                                      x['datetime'] + pd.DateOffset(1) 
                                                      if x['hour_h']< 11 
                                                      else x['datetime'] + pd.DateOffset(2),
                                                      axis=1)
                                              )
        
        return historical_weather
    
    def create_forecast_weather_features(self, forecast_weather):
        '''🔮🌤️ Create forecast weather features 🌤️🔮'''
        
        # Rename column and drop
        forecast_weather = (forecast_weather
                            .rename(columns = {'forecast_datetime': 'datetime'})
                            .drop(columns = 'origin_datetime') # not needed
                           )
        
        # To datetime
        forecast_weather['datetime'] = (pd.to_datetime(forecast_weather['datetime'])
                                        .dt
                                        .tz_localize(None)
                                       )

        # Add county
        forecast_weather[self.lat_lon_columns] = forecast_weather[self.lat_lon_columns].astype(float).round(1)
        forecast_weather = forecast_weather.merge(location, how = 'left', on = self.lat_lon_columns)
        
        # Modify column names - specify suffix
        forecast_weather = self.create_new_column_names(forecast_weather,
                                                        suffix='_f',
                                                        columns_no_change = self.lat_lon_columns + self.weather_join
                                                        ) 
        
        # Group by & calculate aggregate stats 
        agg_columns = [col for col in forecast_weather.columns if col not in self.lat_lon_columns + self.weather_join]
        agg_dict = {agg_col: self.agg_stats for agg_col in agg_columns}
        forecast_weather = forecast_weather.groupby(self.weather_join).agg(agg_dict).reset_index() 
        
        # Flatten the multi column aggregates
        forecast_weather = self.flatten_multi_index_columns(forecast_weather)     
        return forecast_weather

    def create_electricity_features(self, electricity):
        '''⚡ Create electricity prices features ⚡'''
        # To datetime
        electricity['forecast_date'] = pd.to_datetime(electricity['forecast_date'])
        
        # Test set has 1 day offset
        electricity['datetime'] = electricity['forecast_date'] + pd.DateOffset(1)
        
        # Modify column names - specify suffix
        electricity = self.create_new_column_names(electricity, 
                                                   suffix='_electricity',
                                                   columns_no_change = self.electricity_join
                                                  )             
        return electricity

    def create_gas_features(self, gas):
        '''⛽ Create gas prices features ⛽'''
        # Mean gas price
        gas['mean_price_per_mwh'] = (gas['lowest_price_per_mwh'] + gas['highest_price_per_mwh'])/2
        
        # Modify column names - specify suffix
        gas = self.create_new_column_names(gas, 
                                           suffix='_gas',
                                           columns_no_change = self.gas_join
                                          )       
        return gas
    
    def __call__(self, data, client, historical_weather, forecast_weather, electricity, gas):
        '''Processing of features from all datasets, merge together and return features for dataframe df '''
        # Create features for relevant dataset
        data = self.create_data_features(data)
        client = self.create_client_features(client)
        historical_weather = self.create_historical_weather_features(historical_weather)
        forecast_weather = self.create_forecast_weather_features(forecast_weather)
        electricity = self.create_electricity_features(electricity)
        gas = self.create_gas_features(gas)
        
        # 🔗 Merge all datasets into one df 🔗
        df = data.merge(client, how='left', on = self.client_join)
        df = df.merge(historical_weather, how='left', on = self.weather_join)
        df = df.merge(forecast_weather, how='left', on = self.weather_join)
        df = df.merge(electricity, how='left', on = self.electricity_join)
        df = df.merge(gas, how='left', on = self.gas_join)
        
        # Change columns to categorical for XGBoost
        df[self.category_columns] = df[self.category_columns].astype('category')
        return df
    
    
def create_revealed_targets_train(data, N_day_lags):
    '''🎯 Create past revealed_targets for train set based on number of day lags N_day_lags 🎯 '''    
    original_datetime = data['datetime']
    revealed_targets = data[['datetime', 'prediction_unit_id', 'is_consumption', 'target']].copy()
    
    # Create revealed targets for all day lags
    for day_lag in range(2, N_day_lags+1):
        revealed_targets['datetime'] = original_datetime + pd.DateOffset(day_lag)
        data = data.merge(revealed_targets, 
                          how='left', 
                          on = ['datetime', 'prediction_unit_id', 'is_consumption'],
                          suffixes = ('', f'_{day_lag}_days_ago')
                         )
        
    data.drop('datetime', inplace = True, axis = 1)
    
    return data

In [11]:
%%time
# Create all features
N_day_lags = 15 # Specify how many days we want to go back (at least 2)

FeatureProcessor = FeatureProcessorClass()

data = FeatureProcessor(data = train.copy(),
                      client = client.copy(),
                      historical_weather = historical_weather.copy(),
                      forecast_weather = forecast_weather.copy(),
                      electricity = electricity.copy(),
                      gas = gas.copy(),
                     )

df = create_revealed_targets_train(data.copy(), 
                                  N_day_lags = N_day_lags)

CPU times: user 39.5 s, sys: 14.3 s, total: 53.8 s
Wall time: 53.9 s


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 70 columns):
 #   Column                                    Dtype         
---  ------                                    -----         
 0   county                                    category      
 1   is_business                               category      
 2   product_type                              category      
 3   target                                    float64       
 4   is_consumption                            category      
 5   data_block_id                             category      
 6   row_id                                    int64         
 7   prediction_unit_id                        int64         
 8   date                                      datetime64[ns]
 9   year                                      int32         
 10  quarter                                   int32         
 11  month                                     int32         
 12  week          

In [ ]:
for column in df.select_dtypes(include=['float64']).columns:
    df[column].fillna(df[column].mean(), inplace=True)

/tmp/ipykernel_33/1545383620.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mean(), inplace=True)


In [14]:
df

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,date,year,quarter,month,week,hour,day_of_year,day_of_month,day_of_week,eic_count_client,installed_capacity_client,date_client,temperature_h_mean,dewpoint_h_mean,rain_h_mean,snowfall_h_mean,surface_pressure_h_mean,cloudcover_total_h_mean,cloudcover_low_h_mean,cloudcover_mid_h_mean,cloudcover_high_h_mean,windspeed_10m_h_mean,winddirection_10m_h_mean,shortwave_radiation_h_mean,direct_solar_radiation_h_mean,diffuse_radiation_h_mean,hour_h,hours_ahead_f_mean,temperature_f_mean,dewpoint_f_mean,cloudcover_high_f_mean,cloudcover_low_f_mean,cloudcover_mid_f_mean,cloudcover_total_f_mean,10_metre_u_wind_component_f_mean,10_metre_v_wind_component_f_mean,direct_solar_radiation_f_mean,surface_solar_radiation_downwards_f_mean,snowfall_f_mean,total_precipitation_f_mean,forecast_date_electricity,euros_per_mwh_electricity,origin_date_electricity,forecast_date_gas,lowest_price_per_mwh_gas,highest_price_per_mwh_gas,origin_date_gas,mean_price_per_mwh_gas,target_2_days_ago,target_3_days_ago,target_4_days_ago,target_5_days_ago,target_6_days_ago,target_7_days_ago,target_8_days_ago,target_9_days_ago,target_10_days_ago,target_11_days_ago,target_12_days_ago,target_13_days_ago,target_14_days_ago,target_15_days_ago
0,0,0,1,0.713,0,0,0,0,2021-09-01,2021,3,9,35,0,244,1,2,73.426186,1452.343485,NaN,5.426153,1.869131,0.047177,0.016682,1007.663332,62.22133,48.15229,35.164745,36.099351,4.369166,198.540168,105.024747,62.507942,42.516805,11.495674,33.962419,5.442610,1.982612,0.393865,0.444305,0.362350,0.687838,1.099468,0.715676,148.096072,110.240224,0.000026,0.000079,NaT,157.421772,NaN,NaN,95.462096,108.303847,NaN,101.882971,274.759282,274.704243,274.672026,274.617097,274.525302,274.504721,274.411723,274.330178,274.259483,274.208926,274.087474,273.924275,273.774554,273.713833
1,0,0,1,96.590,1,0,1,0,2021-09-01,2021,3,9,35,0,244,1,2,73.426186,1452.343485,NaN,5.426153,1.869131,0.047177,0.016682,1007.663332,62.22133,48.15229,35.164745,36.099351,4.369166,198.540168,105.024747,62.507942,42.516805,11.495674,33.962419,5.442610,1.982612,0.393865,0.444305,0.362350,0.687838,1.099468,0.715676,148.096072,110.240224,0.000026,0.000079,NaT,157.421772,NaN,NaN,95.462096,108.303847,NaN,101.882971,274.759282,274.704243,274.672026,274.617097,274.525302,274.504721,274.411723,274.330178,274.259483,274.208926,274.087474,273.924275,273.774554,273.713833
2,0,0,2,0.000,0,0,2,1,2021-09-01,2021,3,9,35,0,244,1,2,73.426186,1452.343485,NaN,5.426153,1.869131,0.047177,0.016682,1007.663332,62.22133,48.15229,35.164745,36.099351,4.369166,198.540168,105.024747,62.507942,42.516805,11.495674,33.962419,5.442610,1.982612,0.393865,0.444305,0.362350,0.687838,1.099468,0.715676,148.096072,110.240224,0.000026,0.000079,NaT,157.421772,NaN,NaN,95.462096,108.303847,NaN,101.882971,274.759282,274.704243,274.672026,274.617097,274.525302,274.504721,274.411723,274.330178,274.259483,274.208926,274.087474,273.924275,273.774554,273.713833
3,0,0,2,17.314,1,0,3,1,2021-09-01,2021,3,9,35,0,244,1,2,73.426186,1452.343485,NaN,5.426153,1.869131,0.047177,0.016682,1007.663332,62.22133,48.15229,35.164745,36.099351,4.369166,198.540168,105.024747,62.507942,42.516805,11.495674,33.962419,5.442610,1.982612,0.393865,0.444305,0.362350,0.687838,1.099468,0.715676,148.096072,110.240224,0.000026,0.000079,NaT,157.421772,NaN,NaN,95.462096,108.303847,NaN,101.882971,274.759282,274.704243,274.672026,274.617097,274.525302,274.504721,274.411723,274.330178,274.259483,274.208926,274.087474,273.924275,273.774554,273.713833
4,0,0,3,2.904,0,0,4,2,2021-09-01,2021,3,9,35,0,244,1,2,73.426186,1452.343485,NaN,5.426153,1.869131,0.047177,0.016682,1007.663332,62.22133,48.15229,35.164745,36.099351,4.369166,198.540168,105.024747,62.507942,42.516805,11.495674,33.962419,5.442610,1.982612,0.393865,0.444305,0.362350,0.687838,1.099468,0.715676,148.096072,110.240224,0.000026,0.000079,NaT,157.421772,NaN,NaN,95.462096,108.303847,NaN,101.882971,274.759282,274.704243,274.672026,274

In [15]:
# Separating the datetime columns
unnormalized_cols = df.select_dtypes(include=['datetime', 'object', 'category']).columns
df_unnormalized = df[unnormalized_cols]
df_normalized = df.drop(columns=unnormalized_cols)

# Apply MinMaxScaler to non-datetime columns
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_normalized), columns=df_normalized.columns)

# Concatenate the scaled non-datetime data with the datetime columns
df = pd.concat([df_unnormalized, df_normalized], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 70 columns):
 #   Column                                    Dtype         
---  ------                                    -----         
 0   county                                    category      
 1   is_business                               category      
 2   product_type                              category      
 3   is_consumption                            category      
 4   data_block_id                             category      
 5   date                                      datetime64[ns]
 6   date_client                               object        
 7   forecast_date_electricity                 datetime64[ns]
 8   origin_date_electricity                   object        
 9   forecast_date_gas                         object        
 10  origin_date_gas                           object        
 11  target                                    float64       
 12  row_id        

In [16]:
df['date_client'] = pd.to_datetime(df['date_client'])
df['origin_date_electricity'] = pd.to_datetime(df['origin_date_electricity'])
df['forecast_date_gas'] = pd.to_datetime(df['forecast_date_gas'])
df['origin_date_gas'] = pd.to_datetime(df['origin_date_gas'])

# Check the dtypes to confirm the conversion
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 70 columns):
 #   Column                                    Dtype         
---  ------                                    -----         
 0   county                                    category      
 1   is_business                               category      
 2   product_type                              category      
 3   is_consumption                            category      
 4   data_block_id                             category      
 5   date                                      datetime64[ns]
 6   date_client                               datetime64[ns]
 7   forecast_date_electricity                 datetime64[ns]
 8   origin_date_electricity                   datetime64[ns]
 9   forecast_date_gas                         datetime64[ns]
 10  origin_date_gas                           datetime64[ns]
 11  target                                    float64       
 12  row_id        

In [ ]:
target = 'target'
df = df[df[target].notnull()].reset_index(drop=True)

from sklearn.model_selection import KFold

# Define the number of folds
k = 5  # You can choose the number of folds

# Initialize the KFold method
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Use k-fold split
for fold, (train_index, val_index) in enumerate(kf.split(df)):
    # Splitting the data
    tr = df.iloc[train_index]
    val = df.iloc[val_index]

In [ ]:
# Remove columns for features
no_features = ['date', 
                'latitude', 
                'longitude', 
                'data_block_id', 
                'row_id',
                'hours_ahead',
                'hour_h',
                'snowfall_h_mean', 
                'cloudcover_mid_h_mean',
                'windspeed_10m_h_mean',
                'cloudcover_high_h_mean', 
                'rain_h_mean'
               ]

remove_columns = [col for col in df.columns for no_feature in no_features if no_feature in col]
remove_columns.append(target)
features = [col for col in df.columns if col not in remove_columns]
PrintColor(f'There are {len(features)} features: {features}')

There are 54 features: ['county', 'is_business', 'product_type', 'is_consumption', 'prediction_unit_id', 'year', 'quarter', 'month', 'week', 'hour', 'day_of_year', 'day_of_month', 'day_of_week', 'eic_count_client', 'installed_capacity_client', 'temperature_h_mean', 'dewpoint_h_mean', 'surface_pressure_h_mean', 'cloudcover_total_h_mean', 'cloudcover_low_h_mean', 'winddirection_10m_h_mean', 'shortwave_radiation_h_mean', 'direct_solar_radiation_h_mean', 'diffuse_radiation_h_mean', 'temperature_f_mean', 'dewpoint_f_mean', 'cloudcover_high_f_mean', 'cloudcover_low_f_mean', 'cloudcover_mid_f_mean', 'cloudcover_total_f_mean', '10_metre_u_wind_component_f_mean', '10_metre_v_wind_component_f_mean', 'direct_solar_radiation_f_mean', 'surface_solar_radiation_downwards_f_mean', 'snowfall_f_mean', 'total_precipitation_f_mean', 'euros_per_mwh_electricity', 'lowest_price_per_mwh_gas', 'highest_price_per_mwh_gas', 'mean_price_per_mwh_gas', 'target_2_days_ago', 'target_3_days_ago', 'target_4_days_ago', 

In [ ]:
clf = xgb.XGBRegressor(
    device=device,
    enable_categorical=True,
    objective='reg:absoluteerror',
    n_estimators=2 if DEBUG else 1500,
    learning_rate=0.3,  
    early_stopping_rounds=100,
    reg_alpha=5
)

clf.fit(X=tr[features], 
        y=tr[target], 
        eval_set=[(tr[features], tr[target]), (val[features], val[target])],
        verbose=True, 
       )

[0]	validation_0-mae:0.01583	validation_1-mae:0.01586
[1]	validation_0-mae:0.01412	validation_1-mae:0.01415


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=100,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2, n_jobs=None,
             num_parallel_tree=None, objective='reg:absoluteerror', ...)

In [24]:
PrintColor(f'Early stopping on best iteration #{clf.best_iteration} with MAE error on validation set of {clf.best_score:.5f}')

Early stopping on best iteration #1 with MAE error on validation set of 0.01415


In [25]:
# Number of features to display
# DISPLAY = len(features)

# Create a DataFrame with feature names and their importances
importance_data = pd.DataFrame({'name': clf.feature_names_in_, 'importance': clf.feature_importances_})

# Sort the DataFrame by importance in descending order
importance_data = importance_data.sort_values(by='importance', ascending=False)

# Display the top features based on their importance
print(importance_data.head())

                 name  importance
40  target_2_days_ago    0.715185
41  target_3_days_ago    0.093895
45  target_7_days_ago    0.042519
42  target_4_days_ago    0.033080
43  target_5_days_ago    0.027940


In [ ]:
def create_revealed_targets_test(data, previous_revealed_targets, N_day_lags):
    '''🎯 Create new test data based on previous_revealed_targets and N_day_lags 🎯 ''' 
    for count, revealed_targets in enumerate(previous_revealed_targets) :
        day_lag = count + 2
        
        # Get hour
        revealed_targets['hour'] = pd.to_datetime(revealed_targets['datetime']).dt.hour
        
        # Select columns and rename target
        revealed_targets = revealed_targets[['hour', 'prediction_unit_id', 'is_consumption', 'target']]
        revealed_targets = revealed_targets.rename(columns = {"target" : f"target_{day_lag}_days_ago"})
        
        # Add past revealed targets
        data = pd.merge(data,
                        revealed_targets,
                        how = 'left',
                        on = ['hour', 'prediction_unit_id', 'is_consumption'],
                       )
        
    # If revealed_target_columns not available, replace by nan
    all_revealed_columns = [f"target_{day_lag}_days_ago" for day_lag in range(2, N_day_lags+1)]
    missing_columns = list(set(all_revealed_columns) - set(data.columns))
    data[missing_columns] = np.nan 
    
    return data

In [32]:
import enefit
env = enefit.make_env()
iter_test = env.iter_test()

In [33]:
# Reload enefit environment (only in debug mode, otherwise the submission will fail)
if DEBUG:
    enefit.make_env.__called__ = False
    type(env)._state = type(type(env)._state).__dict__['INIT']
    iter_test = env.iter_test()

In [35]:
test

,county,is_business,product_type,is_consumption,datetime,row_id,prediction_unit_id,currently_scored,data_block_id
0,0,False,1,False,1685232000000000000,2005872,0,False,0
1,0,False,1,True,1685232000000000000,2005873,0,False,0
2,0,False,2,False,1685232000000000000,2005874,1,False,0
3,0,False,2,True,1685232000000000000,2005875,1,False,0
4,0,False,3,False,1685232000000000000,2005876,2,False,0
...,...,...,...,...,...,...,...,...,...
3115,15,True,0,True,1685314800000000000,2008987,64,False,0
3116,15,True,1,False,1685314800000000000,2008988,59,False,0
3117,15,True,1,True,1685314800000000000,2008989,59,False,0
3118,15,True,3,False,1685314800000000000,2008990,60,False,0
